In [9]:
# Imports
import numpy as np
import pandas as pd
import os
from utilities import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline    

#加载原始数据
raw_data_train = pd.read_csv(r"./creditGrade_train_data1.csv", encoding='gbk')
raw_data_test = pd.read_csv(r"./test1.csv", encoding='gbk')

#缺失值处理
train_data = raw_data_train.fillna(0)
train_feature_data = (train_data.drop(['id','label1','label2'],axis=1))
train_target = train_data.label2
X_train = train_feature_data.values.astype(np.float32)

test_data = raw_data_test.fillna(0)
test_feature_data = (test_data.drop(['id','label1','label2'],axis=1))
test_target = test_data.label2
X_test = test_feature_data.values.astype(np.float32)

X_train, _, y_train, _ = train_test_split(X_train, train_target, test_size=0.0, random_state=123)
_, X_test, _, y_test = train_test_split(X_test, test_target, test_size=0.9999, random_state=123)

In [10]:
#加载原始数据
#raw_data = pd.read_csv(r"./creditGrade_train_data.csv", encoding='gbk')
#data = raw_data.fillna(0)
#features_data = (data.drop(['id','label1','label2'],axis=1))
#5-class
#target = data.label2
#X = features_data.values.astype(np.float32) # 转换数据类型
#X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.3, random_state=123) # 参数test_size设置训练集占比

In [11]:
#Normalization
from sklearn import preprocessing
# l2正则化
X_train = preprocessing.normalize(X_train, norm='l2')
X_test = preprocessing.normalize(X_test, norm='l2')
print(X_train)


[[  3.43991183e-02   2.97893211e-03   1.53218051e-02 ...,   0.00000000e+00
    3.50693631e-06   0.00000000e+00]
 [  3.06161586e-04   0.00000000e+00   1.18600659e-01 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  1.51673323e-02   4.86652367e-04   1.56499282e-01 ...,   9.40861274e-03
    0.00000000e+00   0.00000000e+00]
 ..., 
 [  7.51737878e-02   0.00000000e+00   8.05112943e-02 ...,   1.53397571e-03
    3.81251273e-04   5.55729819e-03]
 [  7.40233585e-02   0.00000000e+00   1.52661270e-02 ...,   8.16942716e-04
    0.00000000e+00   1.27239479e-03]
 [  2.78981663e-02   0.00000000e+00   3.80462855e-02 ...,  -1.20898162e-03
    0.00000000e+00  -4.46773656e-02]]


In [12]:
#对分类进行one-hot编码
y_tr = one_hot(y_train.astype(np.int64),5)
y_vld = one_hot(y_test.astype(np.int64),5)
print(X_train.shape)
print(X_test.shape)
print("train num:", len(y_tr))
print("val num:", len(y_vld))

(4509, 106)
(810, 106)
train num: 4509
val num: 810


In [13]:
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

#设置相关参数
batch_size = 4500     # Batch size
features_num = 106        # Number of steps
learning_rate = 0.00025
epochs = 5000
n_classes = 5
lambda1 = 0.003
log_dir = r'./logs'    # 输出日志保存的路径

In [14]:
graph = tf.Graph()
# Construct placeholders
with graph.as_default():
    in_units = 106
    h1_units = 900
    h2_units = 1024
    #第一层全连接层参数
    W1 = tf.Variable(tf.truncated_normal([in_units, h1_units], stddev=0.1))
    tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(lambda1)(W1))
    b1 = tf.Variable(tf.zeros([h1_units]))
    
    #第二层卷积层参数
    Wconv1 = tf.Variable(tf.truncated_normal([3, 3, 1, 32], stddev=0.1))
    tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(lambda1)(Wconv1))
    bconv1 = tf.Variable(tf.constant(0.1, shape=[32]))
    
    #第三层卷积层参数
    Wconv2 = tf.Variable(tf.truncated_normal([3, 3, 32, 256], stddev=0.1))
    tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(lambda1)(Wconv2))
    bconv2 = tf.Variable(tf.constant(0.1, shape=[256]))
    
    Wconv3 = tf.Variable(tf.truncated_normal([3, 3, 256, 256], stddev=0.1))
    tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(lambda1)(Wconv3))
    bconv3 = tf.Variable(tf.constant(0.1, shape=[256]))
    
    
    # import1
    #第四层全连接层参数
    #W2 = tf.Variable(tf.truncated_normal([15*15*32, h2_units], stddev=0.1))
    #tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(lambda1)(W2))
    #b2 = tf.Variable(tf.zeros([h2_units]))
    
    #第五层全连接层参数
    #W3 = tf.Variable(tf.zeros([h2_units, 5]))
    #tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(lambda1)(W3))
    #b3 = tf.Variable(tf.zeros([5]))    

    W_att = tf.Variable(tf.truncated_normal([256, 1], stddev=0.1))
    tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(lambda1)(W_att))
    b_att = tf.Variable(tf.zeros([1]))
           
    W_tmp = tf.Variable(tf.truncated_normal([256, 512], stddev=0.1))
    tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(lambda1)(W_tmp))
    b_tmp = tf.Variable(tf.zeros([512]))
        
    W_o = tf.Variable(tf.truncated_normal([512, 5], stddev=0.1))
    tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(lambda1)(W_o))
    b_o = tf.Variable(tf.zeros([5]))
    
    #构造网络
    inputs_ = tf.placeholder(tf.float32, [None, 106], name = 'inputs')
    labels_ = tf.placeholder(tf.float32, [None, 5], name = 'labels')
    keep_prob_ = tf.placeholder(tf.float32, name = 'prob')
    learning_rate_ = tf.placeholder(tf.float32, name = 'learning_rate')

    inputs_ = tf.nn.dropout(inputs_, keep_prob_)
    #第一层全连接层，将维度拓展到900
    hidden1 = tf.nn.relu(tf.matmul(inputs_, W1) + b1)
    hidden1_drop = tf.nn.dropout(hidden1, keep_prob_)
    
    #将数据转换为2维，送入第二层卷积层
    hidden1_drop = tf.reshape(hidden1_drop, [-1, 30, 30, 1])
    h_conv1 = tf.nn.relu(tf.nn.conv2d(hidden1_drop, Wconv1, strides=[1, 1, 1, 1], padding='SAME') + bconv1)
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    h_pool1 = tf.nn.dropout(h_pool1, keep_prob_)
    
    #第三层卷积层
    h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, Wconv2, strides=[1, 1, 1, 1], padding='SAME') + bconv2)
    #h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')    
    #h_pool2 = tf.reshape(h_pool2, [-1, 5*5*64])  #将数据转换为1维
    h_pool2 = tf.nn.dropout(h_conv2, keep_prob_)
 
    # 15*15*32矩阵投影
    h_pro = tf.nn.relu(tf.nn.conv2d(h_pool2, Wconv3, strides=[1, 1, 1, 1], padding='SAME') + bconv3)
    h_pro = tf.reshape(h_pool2, [-1, 256])
    
    # alpha计算
    alpha = tf.reshape(tf.matmul(h_pro, W_att)+b_att, [-1, 15*15])
    alpha = tf.nn.softmax(alpha)
    
    proj = tf.reshape(h_pool2, [-1, 15*15, 256])
    proj = tf.reduce_sum(proj * tf.expand_dims(alpha, 2), 1)  # (N, D)
    print("proj shape:", proj.shape)
 
    proj = tf.reshape(proj, [-1, 256])
    output_o = tf.nn.relu(tf.matmul(proj, W_tmp) + b_tmp)
    print("output_o shape:", output_o.shape)
        
    output_ = tf.matmul(output_o, W_o) + b_o
    print("output shape:", output_.shape)
    
    # import2
    #h_pool2 = tf.reshape(h_pool2, [-1, 15*15*32])  #将数据转换为1维
    #第四层全连接层，将维度转换为500
    #hidden2 = tf.nn.relu(tf.matmul(h_pool2, W2) + b2)
    #hidden2_drop = tf.nn.dropout(hidden2, keep_prob_)
    
    #第五层全连接层，维度转换为5，进行5分类
    #output_ = tf.matmul(hidden2_drop, W3) + b3
    
    # Cost function and optimizer
    #二次代价函数，计算预测值与真实值之间的误差代价值-loss，其中第一个参数logits为最后一层输出，第二个为训练目标值即分类值
    #先通过Softmax函数，输出X对应输出每一类的概率大小，其次和真实值进行“交叉熵”，最终，对向量求均值，得到代价loss
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output_, labels=labels_))
    tf.add_to_collection('losses', loss)
    cost = tf.add_n(tf.get_collection('losses'))
    #cost = loss
    
    #梯度下降法，数据量选择AdamOptimizer
    optimizer = tf.train.AdamOptimizer(learning_rate_).minimize(cost)
    
    # Accuracy
    #correct_pred 返回一个布尔型数组，通过转化为0-1值后来计算准确率
    correct_pred = tf.equal(tf.argmax(output_, 1), tf.argmax(labels_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')
    #预测值
    pred = tf.argmax(output_, 1)
    #正确值
    label = tf.argmax(labels_, 1)

tf.summary.scalar("accuracy", accuracy)
tf.summary.scalar('loss_function', cost)

proj shape: (?, 256)
output_o shape: (?, 512)
output shape: (?, 5)


<tf.Tensor 'loss_function:0' shape=() dtype=string>

In [15]:
if (os.path.exists(r'./checkpoints') == False):
    !mkdir checkpoints
    

In [ ]:
validation_acc = []
validation_loss = []

train_acc = []
train_loss = []

with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
    # summaries合并
    merged = tf.summary.merge_all()    
    # 写到指定的磁盘路径中
    train_writer = tf.summary.FileWriter(log_dir + '/train', sess.graph)
    test_writer = tf.summary.FileWriter(log_dir + '/test')
    # Loop over epochs
    for e in range(epochs):
        
        # Loop over batches
        for x,y in get_batches(X_train, y_tr, batch_size):
            x = x.reshape((batch_size,features_num))
            # Feed dictionary
            feed = {inputs_ : x, labels_ : y,keep_prob_ : 0.75, learning_rate_ : learning_rate}
             # Loss
            summary_str, loss, _ , acc = sess.run([merged, cost, optimizer, accuracy], feed_dict = feed)
         
            train_acc.append(acc)
            train_loss.append(loss)
            # Print at each 5 iters
            if (iteration % 5 == 0):
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {:d}".format(iteration),
                      "Train loss: {:6f}".format(loss),
                      "Train acc: {:.6f}".format(acc))
                train_writer.add_summary(summary_str, e);
            # Compute validation loss at every 10 iterations
            if (iteration%10 == 0):                
                val_acc_ = []
                val_loss_ = []
                
                x_v = X_test.reshape(-1, features_num)
                y_v = y_vld
                feed = {inputs_ : x_v, labels_ : y_v, keep_prob_ : 1.0} 
                summary, loss_v, acc_v, pred_labels = sess.run([merged, cost, accuracy, pred], feed_dict = feed)                    
                val_acc_.append(acc_v)
                val_loss_.append(loss_v)
                
                #for x_v, y_v in get_batches(X_test, y_vld, batch_size):
                    #x_v = x_v.reshape(batch_size, features_num)
                    # Feed
                    #feed = {inputs_ : x_v, labels_ : y_v, keep_prob_ : 1.0}                      
                    # Loss
                    #summary, loss_v, acc_v, pred_labels = sess.run([merged, cost, accuracy, pred], feed_dict = feed)                    
                    #val_acc_.append(acc_v)
                    #val_loss_.append(loss_v)
                    
                # Print info
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {:d}".format(iteration),
                      "Validation loss: {:6f}".format(np.mean(val_loss_)),
                      "Validation acc: {:.6f}".format(np.mean(val_acc_)))
                test_writer.add_summary(summary, e);
                # Store
                validation_acc.append(np.mean(val_acc_))
                validation_loss.append(np.mean(val_loss_))
            # Iterate 
            iteration += 1    
            
        
#     # 保存二进制模型
#     output_graph_def = tf.graph_util.convert_variables_to_constants(sess, sess.graph_def, output_node_names=['labels'])
#     with tf.gfile.FastGFile(r'D:\py_projects\ML\ML_Demo\financial\creditGrade.pb', mode='wb') as f:
#         f.write(output_graph_def.SerializeToString())

    saver.save(sess,"checkpoints-cnn/creditGrade.ckpt")

In [ ]:
t = np.arange(iteration-1)
#print(np.array(train_loss))
plt.figure(figsize = (9,6))
plt.plot(t, np.array(train_loss), 'r-', t[t % 10 == 0], np.array(validation_loss), 'b*')
plt.xlabel("iteration")
plt.ylabel("Loss")
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Plot Accuracies
plt.figure(figsize = (9,6))

plt.plot(t, np.array(train_acc), 'r-', t[t % 10 == 0], validation_acc, 'b*')
plt.xlabel("iteration")
plt.ylabel("Accuray")
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
test_acc = []
pred_labels = []
label = []
with tf.Session(graph=graph) as sess:
    # Restore
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints-cnn'))
    
    x_v = X_test.reshape(-1, features_num)
    y_v = y_vld
    feed = {inputs_ : x_v, labels_ : y_v, keep_prob_ : 1.0} 
    batch_acc = sess.run(accuracy, feed_dict=feed)
    preds = sess.run(pred, feed_dict=feed)
    labels = sess.run(label, feed_dict=feed)
    #max_index = np.argmax(prediction)
    #print(max_index)
    test_acc.append(batch_acc)
    pred_labels.append(preds)
    label.append(labels)
    
    #for x_t, y_t in get_batches(X_test, y_vld, batch_size):
        #x_t = x_t.reshape((batch_size, features_num))
        #feed = {inputs_: x_t,
        #        labels_: y_t,
        #        keep_prob_: 1}
       
        #batch_acc = sess.run(accuracy, feed_dict=feed)
        #preds = sess.run(pred, feed_dict=feed)
        #labels = sess.run(label, feed_dict=feed)
        #test_acc.append(batch_acc)
        #pred_labels.append(preds)
        #label.append(labels)
    print("Test accuracy: {:.6f}".format(np.mean(test_acc)))
    print("pred value", pred_labels)